In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

import matplotlib.pyplot as plt
import random

import librosa
import math

In [ ]:
# path to json
DATA_PATH = "/content/drive/MyDrive/datasets/Data/genres_original/data_10.json"

In [ ]:

def load_data(data_path):

    with open(data_path, "r") as f:
        data = json.load(f)

    # convert lists to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print("Data succesfully loaded!")

    return  X, y


In [ ]:
# load data
X, y = load_data(DATA_PATH)

Data succesfully loaded!


In [ ]:
# create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# create train, validation and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2)

# add an axis to input sets
X_train = X_train[..., np.newaxis]
X_validation = X_validation[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
X_train.shape


(5997, 130, 13, 1)

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)


In [ ]:
# build the CNN
model_cnn = keras.Sequential()

# 1st conv layer
model_cnn.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_cnn.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model_cnn.add(keras.layers.BatchNormalization())

# 2nd conv layer
model_cnn.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model_cnn.add(keras.layers.BatchNormalization())

# 3rd conv layer
model_cnn.add(keras.layers.Conv2D(128, (2, 2), activation='relu'))
model_cnn.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
model_cnn.add(keras.layers.BatchNormalization())

# flatten output and feed it into dense layer
model_cnn.add(keras.layers.Flatten())
model_cnn.add(keras.layers.Dense(256, activation='relu'))
model_cnn.add(keras.layers.Dropout(0.3))

# output layer
model_cnn.add(keras.layers.Dense(10, activation='softmax'))


In [ ]:

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model_cnn.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model_cnn.summary()


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_41 (Conv2D)          (None, 128, 11, 32)       320       
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 64, 6, 32)        0         
 g2D)                                                            
                                                                 
 batch_normalization_3 (Batc  (None, 64, 6, 32)        128       
 hNormalization)                                                 
                                                                 
 conv2d_42 (Conv2D)          (None, 62, 4, 64)         18496     
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 31, 2, 64)        0         
 g2D)                                                            
                                                     

In [ ]:

# train model
history = model_cnn.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=50)

Epoch 1/50
188/188 [==============================] - 13s 60ms/step - loss: 1.8590 - accuracy: 0.3882 - val_loss: 1.4318 - val_accuracy: 0.4953
Epoch 2/50
188/188 [==============================] - 11s 59ms/step - loss: 1.2687 - accuracy: 0.5501 - val_loss: 1.1413 - val_accuracy: 0.6100
Epoch 3/50
188/188 [==============================] - 11s 58ms/step - loss: 1.0537 - accuracy: 0.6221 - val_loss: 1.0682 - val_accuracy: 0.6360
Epoch 4/50
188/188 [==============================] - 10s 54ms/step - loss: 0.8927 - accuracy: 0.6850 - val_loss: 0.9895 - val_accuracy: 0.6620
Epoch 5/50
188/188 [==============================] - 11s 58ms/step - loss: 0.7788 - accuracy: 0.7260 - val_loss: 0.9495 - val_accuracy: 0.6667
Epoch 6/50
188/188 [==============================] - 12s 65ms/step - loss: 0.6852 - accuracy: 0.7619 - val_loss: 0.9298 - val_accuracy: 0.6847
Epoch 7/50
188/188 [==============================] - 11s 59ms/step - loss: 0.5907 - accuracy: 0.8044 - val_loss: 0.9159 - val_accuracy: